In [137]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv("census2021.csv", encoding='latin-1')

# Clean column names
df_raw.columns = df_raw.columns.str.strip()

In [138]:
df_raw['CHARACTERISTIC_NAME']= df_raw['CHARACTERISTIC_NAME'].str.strip()

In [139]:
vars_needed = [
    'Population, 2021',
    'Median age of the population',
    'Median total income of household in 2020 ($)',
    "Bachelor's degree or higher",
    'Employed',
    'Employee',
    'Permanent position',
    'Temporary position',
    'With children',
    'Couple-family households',
    'Car, truck or van - as a driver',
    'Car, truck or van - as a passenger'
]

df_filtered = df_raw[df_raw['CHARACTERISTIC_NAME'].isin(vars_needed)].copy()

In [140]:
# Remove commas and special symbols
df_filtered['C1_COUNT_TOTAL'] = (
    df_filtered['C1_COUNT_TOTAL']
    .replace({',': '', '…': np.nan}, regex=True)
)

df_filtered['C10_RATE_TOTAL'] = (
    df_filtered['C10_RATE_TOTAL']
    .replace({',': '', '…': np.nan}, regex=True)
)

df_filtered['C1_COUNT_TOTAL'] = pd.to_numeric(df_filtered['C1_COUNT_TOTAL'], errors='coerce')
df_filtered['C10_RATE_TOTAL'] = pd.to_numeric(df_filtered['C10_RATE_TOTAL'], errors='coerce')

In [141]:
df_filtered['clean_value'] = df_filtered['C1_COUNT_TOTAL']

# If count is missing (like bachelor's %), use rate instead
df_filtered.loc[
    df_filtered['clean_value'].isna(),
    'clean_value'
] = df_filtered['C10_RATE_TOTAL']

In [218]:
# 1. Define the IDs you want to keep (2024 is your Bachelor's target)
# Add other IDs here (e.g., 1 is usually Population, 6 is Dwellings)
target_ids = [1, 40, 84, 102, 243, 2024, 2225, 2240, 2241, 2242, 2605, 2606] 

# 2. Filter for those IDs
df_target = df_filtered[df_filtered['CHARACTERISTIC_ID'].isin(target_ids)].copy()

# 3. Pivot
df_wide = df_target.pivot(
    index='GEO_NAME', 
    columns='CHARACTERISTIC_NAME', 
    values='clean_value'
)

In [219]:
### Calculating Pop Density

In [220]:
import geopandas as gpd

gdf = gpd.read_file("lfsa000b21a_e.shp")

In [221]:
print(gdf.columns)

Index(['CFSAUID', 'DGUID', 'PRUID', 'PRNAME', 'LANDAREA', 'geometry'], dtype='str')


In [222]:
gdf = gdf.merge(
    df_wide,
    left_on='CFSAUID',
    right_on='GEO_NAME',
    how='left'
)

In [223]:
gdf = gdf.to_crs(epsg=3347)   # Statistics Canada Lambert

In [224]:
gdf['population_density'] = gdf['Population, 2021'] / gdf['LANDAREA']

In [225]:
df_model = gdf.drop(columns='geometry')

In [226]:
df_model.drop(columns=['DGUID','PRUID','PRNAME'], inplace=True)

In [227]:
ontario_fsa = df_model[df_model['CFSAUID'].str[0].isin(['K', 'L', 'M', 'N', 'P'])]

In [228]:
ontario_fsa.reset_index(drop=True, inplace=True)

In [229]:
ontario_fsa.rename(columns={'CFSAUID': 'FSA'}, inplace=True)

In [230]:
ontario_fsa.to_csv('ontario_fsa.csv', index=False)

In [231]:
ontario_fsa.head()

,FSA,LANDAREA,Bachelor's degree or higher,"Car, truck or van - as a driver","Car, truck or van - as a passenger",Couple-family households,Employed,Employee,Median age of the population,Median total income of household in 2020 ($),Permanent position,"Population, 2021",Temporary position,With children,population_density
0,K0A,3072.8212,16495.0,33465.0,2375.0,27880.0,57425.0,51445.0,44.0,115000.0,44510.0,111626.0,6935.0,2655.0,36.326878
1,K0B,1223.8974,1700.0,6985.0,370.0,5200.0,9985.0,8670.0,50.4,79000.0,7500.0,21020.0,1165.0,700.0,17.174642
2,K0C,2470.9420,4200.0,17335.0,1065.0,12980.0,24540.0,22125.0,48.4,84000.0,18915.0,52838.0,3210.0,1155.0,21.383748
3,K0E,1979.2436,2970.0,13315.0,730.0,9870.0,17800.0,16425.0,49.6,83000.0,13990.0,39649.0,2430.0,870.0,20.032400
4,K0G,2675.2361,4445.0,11660.0,860.0,10170.0,18325.0,16145.0,50.4,94000.0,13755.0,39862.0,2395.0,720.0,14.900367
